Import various libraries

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from pywt import wavedec
import pywt
import numpy as np
from scipy.signal import lfilter
import pandas as pd
import scipy.signal as signal
from scipy.stats import iqr
import sys
import copy

import pdb

FIR filter coefficients for HPF and LPF generated using matlab. These are equiripple filters with cutoff frequencies of 3/5Hz and 38/42Hz respectively. Both are of order 30.

In [2]:
#generate FIR filter coeffs using matlab
filters = {}
filters['b_hpf_5hz'] = [-0.197867802488742, -0.0223111834022511, -0.0233980992556299, -0.0245591620459658, -0.0255409709438292, -0.0264980119455013, -0.0273219663603278, -0.0282661633639305, -0.0290666320999297, -0.0295497279590883, -0.0301521603512992, -0.0306588019952879, -0.0309578031663793, -0.0312049403207207, -0.03134442914648, 0.968527520520961, -0.03134442914648, -0.0312049403207207, -0.0309578031663793, -0.0306588019952879, -0.0301521603512992, -0.0295497279590883, -0.0290666320999297, -0.0282661633639305, -0.0273219663603278, -0.0264980119455013, -0.0255409709438292, -0.0245591620459658, -0.0233980992556299, -0.0223111834022511, -0.197867802488742]
filters['b_lpf_40hz'] = [0.0468130734890904, 0.146455226396587, -0.0317230935630268, -0.00916483236202123, -0.0333859647512817, -0.0117562031113877, 0.0200686108179674, 0.0409236312368036, 0.0257162788640763, -0.0205874893332207, -0.063048991609236, -0.0566820018428517, 0.0207298678589156, 0.147289624014856, 0.264830218893494, 0.312523688522207, 0.264830218893494, 0.147289624014856, 0.0207298678589156, -0.0566820018428517, -0.063048991609236, -0.0205874893332207, 0.0257162788640763, 0.0409236312368036, 0.0200686108179674, -0.0117562031113877, -0.0333859647512817, -0.00916483236202123, -0.0317230935630268, 0.146455226396587, 0.0468130734890904]

NUM_BITS_DATA = 12       #define the number of bits in the original data is stored as
FS = 100                 #define the sampling rate
NUM_SAMPLES_BLOCK = 1000 #define the number of samples of data to compress in each block

#don't allow the PRD to be greater than 5%
MAX_PRD = 0.05

#enable/disable plotting
do_plot = False

Function to do wavelet decomposition

In [3]:
def wavelet_decomposition(sig):
    cA2, cD2, cD1 = wavedec(sig, 'db4', level=2)
    coeffs = {'cA2': cA2, 'cD2': cD2, 'cD1': cD1}

    return coeffs

Function to do wavelet reconstruction given the approximation and detail coefficients

In [4]:
def wavelet_reconstruction(coeffs):
    reconstructed = pywt.waverec([coeffs['cA2'], coeffs['cD2'], coeffs['cD1']], 'db4')

    return reconstructed

Function to scale wavelet coefficients to +/- 1. This makes the quantization step consistent regardless of scaling. Return the scaling factors to re-scale to the original range later.

In [5]:
def scale_coeffs(coeffs):
    coeffs_scaled = {}
    scaling_factors = {}

    for key in coeffs.keys():
        tmp_scaling_factor = np.max(np.abs(coeffs[key]))
        tmp_scaled_coeffs = coeffs[key]/tmp_scaling_factor

        scaling_factors[key] = tmp_scaling_factor
        coeffs_scaled[key] = tmp_scaled_coeffs

    return coeffs_scaled, scaling_factors

Function to unscale the wavelet coefficients, given the scaling factor and number of bits 

In [6]:
def unscale_coeffs(coeffs, scaling_factors, bits):
    coeffs_unscaled = {}

    for key in coeffs.keys():
        tmp_coeffs_unscaled = coeffs[key]*scaling_factors[key]/(2**(bits-1))

        coeffs_unscaled[key] = tmp_coeffs_unscaled

    return coeffs_unscaled

Function to combine all the different wavelet coefficients into a single array. This is to simplify the compression step.

In [7]:
def combine_coefficients(coeffs):
    coeffs_combined = []

    #add in each array to coeffs_combined
    coeffs_combined.extend(coeffs['cA2'])
    coeffs_combined.extend(coeffs['cD2'])
    coeffs_combined.extend(coeffs['cD1'])

    #create a mapping of where there are values of zero or nonzero
    #this is the "binary map"
    coeffs_combined = np.array(coeffs_combined)
    binary_map = copy.deepcopy(coeffs_combined)
    binary_map[np.where(binary_map!=0)[0]] = 1

    #drop zero indices
    zero_inds = np.where(coeffs_combined==0)[0]
    coeffs_combined = np.delete(coeffs_combined, zero_inds)

    return coeffs_combined, binary_map

Function to re-map the array of wavelet coefficients back to the original format with zeros, based on the binary map. Also put them back into their respective wavelet decompositions.

In [8]:
def remap_coeffs(coeffs, binary_map, coeff_lengths):
    coeffs_remapped = np.zeros(len(binary_map))
    coeffs_remapped[binary_map==1] = coeffs

    wavelet_remapped = {}
    wavelet_remapped['cA2'] = coeffs_remapped[0:coeff_lengths['cA2']]
    wavelet_remapped['cD2'] = coeffs_remapped[coeff_lengths['cA2']:coeff_lengths['cA2']+coeff_lengths['cD2']]
    wavelet_remapped['cD1'] = coeffs_remapped[coeff_lengths['cA2']+coeff_lengths['cD2']:]

    return wavelet_remapped

Function to convert the quantized coefficients (which are integers, max 8 bit signed) to ascii characters for LZW compression. If it's a signed number, shift up by num_bits to ensure it's positive because ascii values only correspond to positive integers.

In [9]:
def int_to_ascii(array, num_bits=None):
    ascii_str = ''

    #loop through each integer in the array
    for val in array:
        if num_bits is not None:
            #add by 2^(num_bits-1) to make sure all the values are positive
            val = val + 2**(num_bits-1)

        #append the character to the string
        ascii_str = ascii_str + chr(val)

    return ascii_str

Function to convert an ascii string back to an int array

In [10]:
def ascii_to_int(string, num_bits=None):

    if num_bits is None:
        arr = [ord(c) for c in string]
    else:
        arr = [ord(c)-2**(num_bits-1) for c in string]

    return np.array(arr)

Function to compress a string to a list of output symbols using the LZW algorithm

In [11]:
def compress(string):
    #use the extended ascii table
    dict_size = 256

    #build the dictionary
    dictionary = {}
    for i in range(dict_size):
        dictionary[chr(i)] = i

    result = []

    #loop through each character in the string and handle accordingly
    w = ""
    for c in string:
        wc = w + c
        if wc in dictionary:
            w = wc
        else:
            result.append(dictionary[w])
            # add wc to the dictionary
            dictionary[wc] = dict_size
            dict_size += 1
            w = c

    # output the code for w
    if w:
        result.append(dictionary[w])

    return result

Function to decompress a list of output symbols to a string using the LZW algorithm

In [12]:
def decompress(compressed):
    #use the extended ascii table
    dict_size = 256

    #build the dictionary
    dictionary = {}
    for i in range(dict_size):
        dictionary[i] = chr(i)

    w = chr(compressed.pop(0))
    result = [w]

    for k in compressed:
        if k in dictionary:
            entry = dictionary[k]
        elif k == len(dictionary):
            entry = w + w[0]
        else:
            print("Bad compressed")

        result.append(entry)

        dictionary[dict_size] = w + entry[0]
        dict_size += 1

        w = entry

    result = ''.join(result)
    return result

Function to calculate the final compression ratio between the input data and the LZW compressed array. The number of bits in the input signal is calculated as the number of samples times the number of bits per sample. The number of bits in the compressed signal is calculated as the sum of the number of bits of each number in the compressed coefficients and the compressed binary map. Also, each of the scaling coefficients are added as 32 bit floating point numbers.

In [13]:
def calculate_compression_ratio(coeffs_compressed, binary_map_compressed, num_scaling_factors):
    num_bits_compressed = 0

    #loop through each value in the compressed coefficients and compressed binary map
    #and count the number of bits each number uses
    for val in coeffs_compressed:
        num_bits_compressed = num_bits_compressed + len('{0:b}'.format(val))

    for val in binary_map_compressed:
        num_bits_compressed = num_bits_compressed + len('{0:b}'.format(val))

    #assuming floating point values are 32 bits
    num_bits_compressed = num_bits_compressed + 32*num_scaling_factors

    #get the number of bits in the original data
    num_bits_uncompressed = NUM_BITS_DATA*NUM_SAMPLES_BLOCK

    #get the compression ratio
    compression_ratio = num_bits_uncompressed/num_bits_compressed

    return compression_ratio

Function to quantize wavelet coefficients to an integer with a defined number of bits. This is assuming the coefficients have already been scaled to +/- 1. The highest number of bits is 8, and the lowest number of bits is 2. To determine the appropriate nubmer of bits to quanitize the signal, start at 8 bits and then do wavelet reconstruction. Then calculate the percent root mean square difference (PRD) between the reconstructed and original, and keep quantizing with less bits until the PRD is below the MAX_PRD threshold (currently set to 0.05)

In [14]:
def quantize(orig_sig, scaled_coeffs, scaling_factors):
    #starting at 8 bits, keep decreasing the number of bits in the quantization
    #until the PRD is above some threshold. minimum number of bits is 2 since it's
    #a signed integer
    bits = 9

    #initialize PRD to 0 so the while loop can run
    PRD = 0

    if do_plot:
        plt.subplots(figsize=(16,9))
        t = [i*1/FS for i in range(NUM_SAMPLES_BLOCK)]

    while (bits > 1) and (PRD <= MAX_PRD):
        #decrement the number of bits
        bits = bits-1

        #get quantized coefficients
        quantized_coeffs = do_quantization(scaled_coeffs, bits)

        #unscale the data back to the original scale
        unscaled_coeffs = unscale_coeffs(quantized_coeffs, scaling_factors, bits)

        #do wavelet reconstruction on the quantized, unscaled coefficients
        reconstructed_sig = wavelet_reconstruction(unscaled_coeffs)

        #plot stuff
        if do_plot and bits >= 4:
            plt.plot(t, reconstructed_sig, label='Reconstructed @ %i Bits' % bits)

        #calculate PRD
        PRD = calculate_PRD(orig_sig, reconstructed_sig)

    #go back to the previous quantization level if the PRD threshold was crossed
    if PRD > MAX_PRD:
        bits = bits+1
        quantized_coeffs = do_quantization(scaled_coeffs, bits)
        unscaled_coeffs = unscale_coeffs(quantized_coeffs, scaling_factors, bits)
        reconstructed_sig = wavelet_reconstruction(unscaled_coeffs)
        PRD = calculate_PRD(orig_sig, reconstructed_sig)

    #plot some more stuff
    if do_plot:
        plt.plot(t, orig_sig, label='Original Signal')
        plt.title('PRD @ %i Bits = %f' % (bits, PRD))
        plt.legend(loc=1)
        plt.tight_layout()
        plt.xlabel('Time (seconds)')
        plt.ylabel('Amplitude (mV)')
        plt.savefig('figs/PRD.png', dpi=150)
        plt.show()


    return quantized_coeffs, bits, PRD

Helper function to do quantization

In [15]:
def do_quantization(coeffs, bits):
    quantized_coeffs = {}

    for key in coeffs.keys():
        #multiply by 2^(bits-1) because its a signed number
        sig = coeffs[key]
        sig = sig*(2**(bits-1))
        sig = np.round(sig)
        sig = np.array(sig).astype(int)

        quantized_coeffs[key] = sig

    return quantized_coeffs

Function to calcluate the PRD

In [16]:
def calculate_PRD(orig_sig, reconstructed_sig):
    num = np.sum((orig_sig - reconstructed_sig)**2)
    den = np.sum(orig_sig**2)

    PRD = num/den

    return PRD

Main section of the code that loads up data from a pandas dataframe, loops over data in 10 second chunks, performs compression, and then performs decompression.

In [17]:
#load data
df = pd.read_pickle('apnea_ecg.pkl')

#calculate the number of 10 second non-overlapping blocks of data
N = int(len(df)/NUM_SAMPLES_BLOCK)

#calculate the average CR and average PRD
CR_avg = 0
PRD_avg = 0

#loop over the data in 10 second chunks
for i in range(N):
    data = df.ix[i*NUM_SAMPLES_BLOCK:(i+1)*NUM_SAMPLES_BLOCK-1, 0].values

    #subtract out the mean as a precprocessing step
    #for comparison purposes, the mean can be added in after reconstruction
    data = data - np.mean(data)

    #do wavelet decomposition 
    coeffs = wavelet_decomposition(data)

    #scale each set of wavelet coefficients to +/- 1 
    #keep track of the scaling factors to re-scale to the original range later
    coeffs_scaled, scaling_factors = scale_coeffs(coeffs)

    #do quantization
    coeffs_quantized, num_bits, PRD = quantize(data, coeffs_scaled, scaling_factors)
    PRD_avg = PRD_avg + PRD

    #now combine all the coefficients into a single array
    coeffs_combined, binary_map = combine_coefficients(coeffs_quantized)

    #convert the quantized coefficients (which are integers, max 8 bit signed)
    #to ascii characters for LZW compression
    coeffs_ascii = int_to_ascii(coeffs_combined, num_bits)
    binary_map_ascii = int_to_ascii(binary_map)

    #compress and "transmit" the wavelet coefficients and the binary map
    coeffs_compressed = compress(coeffs_ascii)
    binary_map_compressed = compress(binary_map_ascii)

    #calculate the compression ratio
    compression_ratio = calculate_compression_ratio(coeffs_compressed, binary_map_compressed, len(scaling_factors))
    CR_avg = CR_avg + compression_ratio

    #"receive" and uncompress the data
    coeffs_decompressed = decompress(coeffs_compressed)
    binary_map_decompressed = decompress(binary_map_compressed)

    #convert ascii back to int
    coeffs_int = ascii_to_int(coeffs_decompressed, num_bits)
    binary_map_int = ascii_to_int(binary_map_decompressed)

    #re-map the coefficients to the original format with zeros, based on the binary map
    #also put them back into their respective wavelet decompositions
    #before doing the remapping, find out the length of each wavelet approximation and detail coefficients
    #this step isn't necessary if the length of the input signal is fixed
    coeff_lengths = {'cA2': len(coeffs['cA2']), 'cD2': len(coeffs['cD2']), 'cD1': len(coeffs['cD1'])}
    coeffs_remapped = remap_coeffs(coeffs_int, binary_map_int, coeff_lengths)

    #unscale the coefficients
    coeffs_unscaled = unscale_coeffs(coeffs_remapped, scaling_factors, num_bits)

    #do wavelet reconstruction
    data_reconstructed = wavelet_reconstruction(coeffs_unscaled)

    if do_plot:
        t = [i*1/FS for i in range(NUM_SAMPLES_BLOCK)]
        plt.subplots(figsize=(16,9))
        plt.plot(t, data, label='Original Signal')
        plt.plot(t, data_reconstructed, label='Reconstructed Signal')
        plt.title('Compression Ratio: %.1f' % compression_ratio)
        plt.xlabel('Time (seconds)')
        plt.ylabel('Amplitude (mV)')
        plt.tight_layout()
        plt.legend(loc=1)
        plt.savefig('figs/reconstructed.png', dpi=150)
        plt.show()

#calculate the average compression ratio and PRD
CR_avg = CR_avg/N
PRD_avg = PRD_avg/N
print('Average compression ratio: %.3f' % CR_avg)
print('Average PRD: %.3f' % PRD_avg)

/Users/nbobra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  del sys.path[0]


Average compression ratio: 5.633
Average PRD: 0.025
